# 3. Zoeken naar een combinatie van termen
Als je op zoek bent naar samenhang tussen twee zaken wil je testen of de termen die naar die zaken verwijzen inderdaad in combinatie met elkaar voorkomen. Dan moet je bepalen op welke varianten van die twee termen je wilt zoeken en wat de minimale en maximale afstand tussen die termen moet zijn. 

In de onderstaande code kun je op de eerste plaats twee zoektermen opgeven, als waarden van `search_term1` en `search_term2`. De variabele `window` specificeert de maximale afstand tussen deze twee woorden. 

De code gaat vervolgens op zoek naar alle passages van de opgegeven lengte waar de beide zoektermen in voorkomen. De gevonden passages worden in dit notebook getoond wanneer de code wordt uitgevoerd, en de resultaten worden eveneens weggeschreven in een bestand met de naam 'cooccurrence.txt'.

In [1]:
from kitlvTdm import *
import os
from os.path import join
import re
import pandas as pd

In [2]:
# Defineer de zoektermen en de seach window
search_term1 = 'soldaat'
search_term2 = 'geweer'
window = 30

# In deze map gaat er gezocht worden
path = "Corpus"

df = pd.DataFrame(columns = ['count_within_document','file','title','fragment'])

# Het zoeken is hoofdletter gevoelig, daarom worden alle hoofdletters in kleine letters veranderd, hier en in de corpus.
search_term1 = search_term1.lower()
search_term2 = search_term2.lower()

pattern = '####'


# Voor ieder bestand in de eerder gedefineerde map
for file in os.listdir( path ):
    
    # Laat zien welk bestand het programma aan het lezen is
    print( f"Reading {file} ...")
    
    # Maak een lijst aan waar alle fragmenten in komen
    fragments = []
    
    # Als het bestand een tekst bestand is:
    if re.search( 'txt$' , file ):
        
        # Open het bestand en laad de tekst in de variabele 'fullText'
        with open( join( path , file) , encoding = 'utf-8') as egodocument:
            fullText = egodocument.read()
            
        # Zoek en sla alleen de tekst op van iedere bladzijde (bijvoorbeeld niet het bladzijde nummer)
        fullText = re.sub( 'page\s+\d+', '' , fullText )
        
        # Verander alle hoofdletters in kleine letters
        fullText =  fullText.lower()
        
        # Identificeer de losse woorden
        words = tokenise(fullText)
        
        # Voor iedere locatie van een woord
        for i in range( 0 , (len(words)) ):
            
            # creeer een fragment van de grootte die gespecificeerd is in 'window'
            fragment = ' '.join( words[ i : window+i ] )
            
            # als de gezochte woorden beide voorkomen als los woord in het fragment
            if re.search( r'\b{}\b'.format(search_term1) , fragment , re.IGNORECASE ) and re.search( r'\b{}\b'.format(search_term2) , fragment , re.IGNORECASE ):
                
                '''
                Dit stukje zorgt ervoor dat een fragment niet vaker gevonden wordt. Als er één woord tussen soldaat en geweer zit
                en de 'window' is 30 zijn er 27 verschillende fragmenten voor dezelfde gevonden combinatie.
                Om dit te voorkomen wordt er gekeken of in een gevonden fragment de zoektermen op een andere manier tot elkaar staan
                en daarmee ook daadwerkelijk een nieuw fragment zijn.                
                '''
                if not( re.search( pattern , fragment , re.IGNORECASE ) ):
                    
                    # Voeg het gevonden fragment toe aan de lijst fragments
                    fragments.append( fragment )
            
                    # Voor ieder nummer tussen 0 en het nummer 'window'
                    for findex in range( 0 , window ):
                        
                        # Als de het patroon al bekent is stop met de bovenstaande 'for loop'
                        if re.search( r'\b{}\b'.format(search_term1) , words[i + findex] , re.IGNORECASE ) or re.search( r'\b{}\b'.format(search_term2) , words[i + findex] , re.IGNORECASE ):
                            break
                    
                    # Anders update de variabele pattern naar het nieuw gevonden patroon
                    pattern = ' '.join( words[ i + findex : window+i  ] )

            # Tel een op bij de plek waar we zijn bij het doorzoeken van de woorden (bovenste for loop)
            i = window + i
            
    documentCount = 0
    # Als er resultaten gevonden zijn
    if len(fragments) > 0:
        
        # Vind de titel van het boek en sla het op in de variabele 'title'
        title = showTitle(file)
        
        # Laat het volgende zien met de titel en de naam van het bestand
        print(f"{len(fragments)} occurrence(s) in {title} ({file})\n")
        
        # Voor ieder gevonden fragment 
        for fragment in fragments:
            
            print(f"{fragment}\n")
            
            documentCount += 1
            
            # Schrijf het fragment op in het bestand
            df = df.append({'count_within_document' : documentCount,'file' : file, 'title' : title, 'fragment' : fragment}, ignore_index = True)

df.to_excel(f"combination_{search_term1}_{search_term2}.xlsx")
print(f"Excel file saved as combination_{search_term1}_{search_term2}.xlsx")

Reading 03391.txt ...
Reading 03392.txt ...
1 occurrence(s) in De vergeten dagboeken (03392.txt)

werk verricht bij ons komt een jongen aan van 1-11 hij vertelt dat hij een soldaat van de vermiste sectie is door automatisch vuur werd de kolf van zn geweer

Reading 03393.txt ...
4 occurrence(s) in Hij ging, kwam terug en keerde weer (03393.txt)

je kunt wel een stuk lood in je pens krijgen als je wilt je bent een goed soldaat zei hij mij op de schouders kloppend ja had ik mijn geweer

persoonlijk de hand drukken toch wel een teken dat ze je mogen en dit doetje werkelijk goed op een andere post wordt bij het schoonmaken van een geweer de soldaat

angstige inwoners uit het dorp als we uit het gezicht zijn van de vijand rukt korporaal cor mijn geweer uit mijn handen en wil gaan schieten op een t.n soldaat

uit mijn handen en wil gaan schieten op een t.n soldaat die zeker vijfhonderd meter van ons vandaan staat ik pak hem het geweer weer meteen uit de handen met

Reading 03394.txt ...
Readi

Reading 03454.txt ...
Reading 03455.txt ...
Reading 03456.txt ...
1 occurrence(s) in IndiÃ« blijft ons bezighouden (03456.txt)

hoe grote witte vogels zich aan de rijstkorrels te goed deden haalde soldaat hans een kwajongensstreek uit die verstrekkende gevolgen had balsturig vies en niet uitgeslapen pakte hij zijn geweer

Reading 03457.txt ...
3 occurrence(s) in Camouflagepak, politiepet en narrenkap (03457.txt)

in de kleuren rood wit en blauw en voor dat huisje stond een soldaat op wacht althans dat kon je met wat fantasie aannemen hij was bewapend met een geweer

hij bleef ze te allen tijde verbaal de baas een typisch voorbeeld hiervan was zijn tweede degradatie jo had in een dolle bui in het veld het geweer van soldaat

arresteren zou de plopper zijn geweer aangelegd hebben dan was hij het haasje geweest want in de struiken lagen dertig man met een vuurwapen op hem gericht alhoewel de tni-soldaat

Reading 03458.txt ...
Reading 03459.txt ...
Reading 03460.txt ...
Reading 03461.txt .

Reading 03500.txt ...
Reading 03501.txt ...
Reading 03502.txt ...
Reading 03503.txt ...
Reading 03504.txt ...
Reading 03505.txt ...
Reading 03506.txt ...
Reading 03507.txt ...
2 occurrence(s) in Soldaat zonder franje (1928) (03507.txt)

patrouille meegelopen onder leiding van soldaat 1 kl jan havikus op de hellingen van de vulkaan merbaboe op een dag zag een van de jongens in zijn tent een geweer

de opdracht s-nachts gewapend met geweer plus bajonet voor de stellingen de aanvallers op te wachten een zinloze en volkomen idiote opdracht bij het vallen van de avond kreeg soldaat

Reading 03508.txt ...
Reading 03509.txt ...
1 occurrence(s) in Vaarwel, kameraad! : met het vergeten leger in Indie? (03509.txt)

later is er buiten een ontploffing ik weer naar buiten heeft u bet geboord richting gemolong neem je geweer en kom mee terwijl we naar buiten lopen vertelt een soldaat

Reading 03510.txt ...
Reading 03511.txt ...
2 occurrence(s) in Herinneringen aan de 42e Zibva (03511.txt)

was het d

Reading 03562.txt ...
2 occurrence(s) in "Over Unentse soldaten overzee : Den Vaderlant Ghetrouwe"" in Nederlands-Indie 1945-1950 /""" (03562.txt)

pantserautos 59 motoraffuiten 24 ambulanceautos 48 organisatie van een infanteriedivisie bij de bewapening is niet vermeld dat bij de persoonlijke uitrusting van elke soldaat ook nog een lee enfield geweer

een zware prikkeldraadomheining voor aan de straat was een slagboom aangebracht een soldaat uit oirschot was op een snikhete dag de wachtpost hij werd geacht daar staand met het geweer

Reading 03563.txt ...
Reading 03564.txt ...
Reading 03565.txt ...
1 occurrence(s) in Oost-Java : gedenkboek der 4e Infanterie-Brigade (03565.txt)

en zelfs kwamen we tot de ontdekking dat een soldaat flanken heeft de instructeur vertelde ons hoe we op de beste manier konden omgaan met een bren sten of geweer

Reading 03566.txt ...
3 occurrence(s) in Brieven uit Indie (03566.txt)

zo vanaf kwamen ook in de omgeving van salatiga moesten we patrouilleren we 

Reading 03620.txt ...
Reading 03621.txt ...
Reading 03622.txt ...
Reading 03623.txt ...
Reading 03624.txt ...
3 occurrence(s) in Vertel nog eens over de oorlog : oorlogsherinneringen van een onderwijzer (1991) (03624.txt)

deed de deur open en een duitse soldaat komt binnen hij was alleen en zag er vermoeid en angstig uit hij streek neer in een stoel en legde zijn geweer

munitie de stoker van de centrale verwarming leenheer was in het complot betrokken het geweer moest ook eens beproefd worden want aan een kapot geweer heb je niets de soldaat

moest ook eens beproefd worden want aan een kapot geweer heb je niets de soldaat uit de mobilisatie weet daar wel raad op in tegenwoordigheid van leenheer lost hij een

Reading 03625.txt ...
Reading 03626.txt ...
2 occurrence(s) in Een stem uit het veld : herinneringen van de ritmeester-adjudant van generaal S.H. Spoor (1988) (03626.txt)

een gevallen held vervangen aldus generaal loekito santoso wat een moed is nodig om ongewapend met een gewee

Reading 03653.txt ...
Reading 03654.txt ...
Reading 03655.txt ...
Reading 03656.txt ...
Reading 03657.txt ...
Reading 03658.txt ...
Reading 03659.txt ...
2 occurrence(s) in Morotai : de bevrijding van de Grote Oost en Borneo (april 1944-april 1946) (1976) (03659.txt)

gewapende soldaat op post die telkens als er een meningsverschil was met de leiding van de vergadering bij wijze van waarschuwing een paar maal met de kolf van zijn geweer

vijandelijke wapenwerking maar tengevolge van eigen fouten of grove onvoorzichtigheid bij de x-brigade is het gebeurd ik meen tot tweemaal toe dat een soldaat bij het schoonmaken van zijn geweer

Reading 03660.txt ...
Reading 03661.txt ...
1 occurrence(s) in Daar werd iets grootsch verricht', of Hoe het Koninkrijk der Nederlanden zijn grootste kolonie verloor (1980) (03661.txt)

die de walkie-talkie bedienden en die ook niks te schieten hadden gehad in die actie die zien dus dat afscheid het mannetje verwijderde zich plotseling pakt een soldaat een gew

Reading 03756.txt ...
Reading 03757.txt ...
Reading 03758.txt ...
Reading 03759.txt ...
Reading 03760.txt ...
Reading 03761.txt ...
Reading 03762.txt ...
1 occurrence(s) in Zo was het : Gedenkboek 3-5-R.I. 1946-1950 (1950) (03762.txt)

verlaten de geweer makerij deze dienst vergde weinig personeel maar toch hadden we het moeilijk zonder hem kunnen doen de sergeant westerweel begon op 11 november 1946 tesamen met soldaat

Reading 03763.txt ...
2 occurrence(s) in 1e Bataljon, 5e Regiment Infanterie : gedenkboek 1945-1948. (1951) (03763.txt)

ik heb dit geweer daarop medegenomen naar de fourier en het op de rustkamer opgeborgen het geweer is van het type lee enfield met geweernummer av 6553 toebehorende aan soldaat

daarop medegenomen naar de fourier en het op de rustkamer opgeborgen het geweer is van het type lee enfield met geweernummer av 6553 toebehorende aan soldaat p van gelder van

Reading 03764.txt ...
Reading 03765.txt ...
Reading 03766.txt ...
Reading 03767.txt ...
4 occurrence(

Reading 03836.txt ...
Reading 03837.txt ...
1 occurrence(s) in Herinneringen van een gemobiliseerde Infanterist, Engeland - en IndiÃ«ganger 1939-1948 (1991) (03837.txt)

waar ik liep op enige afstand om de hoek links van mij hoorde ik met militaire stap iemand naderbij komen het was een soldaat in grijs-groene uniform met het geweer

Reading 03838.txt ...
2 occurrence(s) in Brief aan een psycholoog: oorlogservaringen (2000) (03838.txt)

trein vanuit ramsgate naar wolverhampton besefte ik pas goed dat er een andere tijd voor mij was aangebroken en ik denk ook voor mijn medesoldaten soldaat ofwel infanterist zonder geweer

k epiloog k er was weer een periode afgesloten in mijn toch wel jonge en veelbewogen leven ik was nu een echte soldaat geworden die wist hoe je een geweer

Reading 03839.txt ...
1 occurrence(s) in Er waren er die niet gingen: Vijftien eeuwen straf voor IndonesiÃ«-weigeraars (1995) (03839.txt)

sergeant en drie europese fuseliers om de verliezen onder de bevolking van b

1 occurrence(s) in De medische geschiedenis van een infanterie-bataljon der Koninklijke Landmacht gedurende drie jaar actieve dienst op Java, 1946-1950 (03984.txt)

aan meer concrete militaire groepen kan ontstaan de training bewerkstelligt verder een binding van de soldaat aan zijn specifieke functie en aan zijn peloton hij raakt vertrouwd met zijn geweer

Reading 03985.txt ...
Reading 03986.txt ...
Reading 03991.txt ...
2 occurrence(s) in TabÃ© Java TabÃ© IndiÃ« (03991.txt)

hoeken let eens goed op die stapel matten en rommel op de bijgevoegde tekening is te zien hoe de soldaat het aan moest pakken met de loop van zijn geweer

soldaat ed zijlemaker dit was typisch het karakter van een guerrillaoorlog het ene ogenblik met een picke soort hakgereedschap aan het werk in de sawah even later met een geweer

Reading 03992.txt ...
1 occurrence(s) in """Loslaten, dat gaat niet meer""." (2017) (03992.txt)

links en rechts achter ook een soldaat en op de achterlijn twee soldaten met de rug naa

NameError: name 'searchterm1' is not defined

**Ofening 3: Ga, met behulp van de bovenstaande code, op zoek naar passages waarin twee relevante zoektermen in voorkomen. Pas de waarden van de variabelen `search_term1` en `search_term2` aan. Indien nodig kun je ook de waarde van `window` aanpassen.**